# Clean Offers

## Load Data

In [1]:
import pandas as pd

fileName = 'Test_Results'
excel_file = './' + fileName + '.xlsx'

df = pd.read_excel(excel_file)

## Translate

In [2]:
from langdetect import detect
from deep_translator import GoogleTranslator

In [3]:
def detect_and_translate(text):
    if pd.isna(text):
        return text
    try:
        detected_language = detect(text)
        if detected_language != 'en':
            translator = GoogleTranslator(source='auto', target='en')
            return translator.translate(text)
        else:
            return text
    except Exception as e:
        return text

In [4]:
columns_to_translate = ['Descriptions', 'Requirements']
df[columns_to_translate] = df[columns_to_translate].applymap(detect_and_translate)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_102584\3137583933.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_translate] = df[columns_to_translate].applymap(detect_and_translate)


# Fix Line Break Issue

In [5]:
import re

def split_words(text, excluded_words=None):
    if not isinstance(text, str):
        return text
    
    if excluded_words is None:
        excluded_words = []

    placeholders = {}
    for i, word in enumerate(excluded_words):
        placeholder = f"__PLACEHOLDER_{i}__"
        placeholders[placeholder] = word
        text = text.replace(word, placeholder)

    def split_non_excluded(match):
        word = match.group(0)
        return re.sub(r'([a-z])([A-Z])|([a-zA-Z])(\d)|(\d)([a-zA-Z])', r'\1\3\5 \2\4\6', word)
    
    splitted_text = re.sub(r'\S+', split_non_excluded, text)
    
    for placeholder, original_word in placeholders.items():
        splitted_text = splitted_text.replace(placeholder, ' ' + original_word)
    
    splitted_text = re.sub(r'(\W)', r' \1 ', splitted_text)
    splitted_text = re.sub(r'\s+', ' ', splitted_text).strip()
    
    return splitted_text

In [6]:
excluded_words = ['PhD', 'phd', 'Phd', 'phD', 'PHd', 'P.h.d', 'P.h.D', 'P.H.D']
columns_to_fix = ['Descriptions', 'Requirements']

df[columns_to_fix] = df[columns_to_fix].applymap(lambda x: split_words(x, excluded_words=excluded_words))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_102584\3629075693.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_fix] = df[columns_to_fix].applymap(lambda x: split_words(x, excluded_words=excluded_words))


## Save Data

In [7]:
df.to_excel('./' + fileName + '_Translated.xlsx', index=False, engine='openpyxl')